# Product Invoice Classification

* Inv_ID (Invoice ID) : Unique number representing Invoice created by supplier/vendor

* Vendor Code (Vendor ID) : Unique number representing Vendor/Seller in the procurement system

* GL_Code: Account’s Reference ID

* Inv_Amt : Invoice Amount

* Item Description : Description of Item Purchased Example: “Corporate Services Human Resources Contingent Labor/Temp Labor Contingent Labor/Temp Labor”

* Product Category : Category of Product for which Invoice is raised A pseudo product category is represented in the dataset as CLASS-???, where ? is a digit.* 

## Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
# for saving the pipeline
import joblib

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.neighbors import KNeighborsClassifier

In [5]:
from sklearn.pipeline import Pipeline

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

In [7]:
from feature_engine.encoding import OrdinalEncoder
from feature_engine.selection import DropFeatures

## Data

In [49]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')
sample_submission_data = pd.read_csv('sample_submission.csv')

## Transformations

In [9]:
## map for the target
tmp = train_data.groupby('Product_Category', as_index = False).agg(counts = ('Inv_Id','count')).sort_values('counts')
tmp['ranks'] = range(1,len(tmp)+1)
map_target = dict(tmp[['Product_Category','ranks']].values)

In [10]:
train_data['Map_Product_Category'] = train_data.Product_Category.map(map_target)

In [11]:
def splitter(data):
    data[['TypeEnt','TypeEnt_number']] = data.Vendor_Code.str.split('-',expand=True)
    data[['GCL','GCL_number']] = data.GL_Code.str.split('-',expand=True)
    return data

In [12]:
train_data = splitter(data = train_data)

###  split data

In [13]:
seed = 12

In [14]:
X = train_data[['Inv_Id', 'Vendor_Code', 'GL_Code', 'Inv_Amt', 'Item_Description']]
y = train_data.Map_Product_Category

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=seed)

### Pipeline

<b> expanding

In [16]:
X_train = splitter(data = X_train)

<b> maps

In [17]:
def get_dictionary_rank1_tomap(data, variable):
    tmp = data.groupby([variable,'Map_Product_Category'], as_index = False).agg(counts = ('Inv_Id','count'))
    tmp["Rank"] = tmp.groupby([variable])["counts"].rank("dense", ascending=False)
    tmp = tmp[tmp.Rank == 1]
    map_rank =  dict(tmp[[variable,'Map_Product_Category']].values)
    return map_rank

In [18]:
TypeEnt_number_maps = get_dictionary_rank1_tomap(data = train_data, variable = 'TypeEnt_number')

In [19]:
# categorical missing value imputer
class Mapper(BaseEstimator, TransformerMixin):

    def __init__(self, variables, mappings):

        if not isinstance(variables, list):
            raise ValueError('variables should be a list')

        self.variables = variables
        self.mappings = mappings

    def fit(self, X, y=None):
        # we need the fit statement to accomodate the sklearn pipeline
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.variables:
            X[feature] = X[feature].map(self.mappings)

        return X

In [81]:
class Custom_Fillna(BaseEstimator, TransformerMixin):
    
    def __init__(self, variables, fill_value):
        
        if not isinstance(variables, list):
            raise ValueError('variables should be a list')
            
        self.variables = variables
        self.fill_value = fill_value
        
    def fit(self, X, y=None):
        # we need the fit statement to accomodate the sklearn pipeline
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.variables:
            X[feature] = X[feature].fillna(self.fill_value)

        return X

In [82]:
to_drop = ['Inv_Id','Vendor_Code','GL_Code', 'Item_Description','TypeEnt','GCL']

In [83]:
category_prediction_pipeline = Pipeline([
    ('GCL_Code-cardinal-ordering',
     OrdinalEncoder(encoding_method='ordered', variables=['GCL_number'])
    ),
    
    ('TypeEnt_number_map_modes', 
    Mapper(variables = ['TypeEnt_number'], mappings = TypeEnt_number_maps)
    ),
    
    ('drop_features',
     DropFeatures(features_to_drop= to_drop)
    ),
    
    ('Fill_na',
    Custom_Fillna(variables = ['TypeEnt_number'], fill_value = 0)
    ),
    
    ('KNN', KNeighborsClassifier(n_neighbors = 4 ))
    
])

In [84]:
category_prediction_pipeline.fit(X_train, y_train)

Pipeline(steps=[('GCL_Code-cardinal-ordering',
                 OrdinalEncoder(variables=['GCL_number'])),
                ('TypeEnt_number_map_modes',
                 Mapper(mappings={'1000': 16, '1003': 35, '1005': 35,
                                  '1006': 35, '1007': 35, '1008': 35,
                                  '1010': 35, '1011': 35, '1012': 35,
                                  '1013': 35, '1014': 35, '1016': 35,
                                  '1018': 35, '1019': 35, '1021': 35,
                                  '1022': 35, '1024': 35, '1025': 35,
                                  '1026': 35, '1027': 35, '1028': 9, '1030': 35,
                                  '1031': 35, '1032': 35, '1033': 35,
                                  '1034': 25, '1035': 29, '1036': 27,
                                  '1037': 35, '1038': 35, ...},
                        variables=['TypeEnt_number'])),
                ('drop_features',
                 DropFeatures(features_to_drop=['Inv_

In [85]:
pred = category_prediction_pipeline.predict(X_train)

In [86]:
pred

array([32, 35, 29, ..., 35, 36, 34], dtype=int64)

<b> validation

In [87]:
X_val = splitter(data = X_val)

In [88]:
X_val.head()

,Inv_Id,Vendor_Code,GL_Code,Inv_Amt,Item_Description,TypeEnt,TypeEnt_number,GCL,GCL_number
2546,18659,VENDOR-1608,GL-6101400,92.93,Capital Southwest Corp Final Site Clean Up May...,VENDOR,1608,GL,6101400
2486,18574,VENDOR-1065,GL-6050310,93.45,Miscellaneous Company Car (Field Only) May2004...,VENDOR,1065,GL,6050310
5218,22496,VENDOR-2008,GL-6100500,61.05,Designatronics Inc Magazines Feb-2002 Media Bu...,VENDOR,2008,GL,6100500
3804,20468,VENDOR-1097,GL-6101400,32.10,General Contractor Store Construction General ...,VENDOR,1097,GL,6101400
3057,19423,VENDOR-2076,GL-6101400,1.60,General Contractor Store Construction General ...,VENDOR,2076,GL,6101400


In [89]:
pred = category_prediction_pipeline.predict(X_val)

In [90]:
pred

array([34, 36, 27, ..., 30, 33, 33], dtype=int64)

<b> test data

In [98]:
test_data = splitter(data = test_data)

In [99]:
X_test = test_data[['Inv_Id', 'Vendor_Code', 'GL_Code', 'Inv_Amt', 'Item_Description']]

In [100]:
X_test = splitter(data = X_test)

C:\Users\Miguel\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [103]:
pred = category_prediction_pipeline.predict(X_test)

In [104]:
pred

array([36, 34, 34, ..., 33, 31, 16], dtype=int64)

## saving machine

In [105]:
joblib.dump(category_prediction_pipeline, 'category_prediction_pipeline.joblib') 

['category_prediction_pipeline.joblib']